# image_processing

In [ ]:
import numpy as np
from nilearn import image
from nilearn.image import get_data
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from arch.bootstrap import IIDBootstrap
from scipy.stats import kurtosis
import re
import time
import json
from collections import Counter
#import dask
#from dask.distributed import Client
#from dask import delayed

In [ ]:
import glob
import pandas as pd
path= '/dicom/'
label_path='with_category.csv'
indicative="000"
df_labels=pd.read_csv(label_path,sep=';')
df_labels['id1']=indicative+df_labels['id'].astype(str)
df_labels['path']=path+df_labels['id1']+'/'

In [ ]:
# skipping studies with previous surgeries or artifacts
skip = ['00020023096','00020029351','00030026189','00030037155','00030037273','00050009584','00050002822',
        '00050004047','00050004076','00050004345','00050004468','00050004755']
df_labels=df_labels[~df_labels.id1.isin(skip)]

##  Get images and collapsing data

In [ ]:
path_health=df_labels[df_labels.label==1].path
path_pathologic=df_labels[df_labels.label==0].path
nx=45;ny=109;nz=91
space = 'mask'
dat_healt=getDataFromDirectoryP(path_health.values,space,nx,ny,nz)
dat_pathologic=getDataFromDirectoryP(path_pathologic.values,space,nx,ny,nz)


In [ ]:
print('health neuroimaging: ',len(dat_healt))
print('pathologic neuroimaging: ',len(dat_pathologic))

# Threshold: Healty VS Pathologic

In [ ]:
W = [.9,.1]
n_boost = 1000
trheshol,check,PH,PP,diff_healt , diff_patholo, diff_healt0 , diff_patholo0 = thersPH(dat_healt,dat_pathologic,W,n_boost)

In [ ]:
print('True postive Diferences without weight: ', np.mean(check['No_weight']),
      '\nTrue postive  Diferences with weight: ',np.mean(check['weight']))

In [ ]:
f, (ax1) = plt.subplots(1, 1,figsize=(10, 5))
sns.distplot(diff_healt0, ax=ax1, kde=True, rug=True, label="Healthy")
sns.distplot(diff_patholo0, ax=ax1, kde=True, rug=True, label="Pathologic")
ax1.set(xlabel=r'$\overline{dp}$', ylabel='Count')
f.legend(bbox_to_anchor=(.5, .7), loc=2, borderaxespad=0., title='Group')
plt.show()

# Permutations between hemispheres test (HCP)

In [ ]:
# checking the random generation
n=nx*ny*nz
check_rangene = []
for i in range(1000):
    cal = permut(n)
    check_rangene.append(cal)

In [ ]:
print('average of unique values per iteration: ',np.mean(check_rangene))

In [ ]:
resul_permut_helth=[]
resul_permut_pathologic=[]
n=nx*ny*nz

In [ ]:
%%time
for i in np.arange(len(dat_healt)):
    res = rPermut(500,dat_healt[i]['left'],dat_healt[i]['right'],1,n,'rate')
    Resh = {'R':res['R'],'label':dat_healt[i]['label'].tolist(),'norm_x':res['norm_x'],'norm_y':res['norm_y']}
    resul_permut_helth.append(Resh)

In [ ]:
%%time
for i in np.arange(len(dat_pathologic)):
    res = rPermut(500,dat_pathologic[i]['left'],dat_pathologic[i]['right'],1,n,'rate')
    Resp = {'R':res['R'],'label':dat_pathologic[i]['label'].tolist(),'norm_x':res['norm_x'],'norm_y':res['norm_y']}
    resul_permut_pathologic.append(Resp)

In [ ]:
print('health neuroimaging test: ',len(resul_permut_helth))
print('pathologic neuroimaging test: ',len(resul_permut_pathologic))

# Save and load data

In [ ]:
# Save healty
dumped_healty = json.dumps(resul_permut_helth, cls=NumpyEncoder) # , sort_keys=False, indent=4

with open('resul_permut_helth_corrected.json', 'w') as f:
    json.dump(dumped_healty, f) # , sort_keys=False, indent=4

In [ ]:
# Save patholigic
dumped_pathologic = json.dumps(resul_permut_pathologic, cls=NumpyEncoder) # , sort_keys=False, indent=4

with open('resul_permut_pathologic_corrected.json', 'w') as f:
    json.dump(dumped_pathologic, f) # , sort_keys=False, indent=4

In [ ]:
# Load data healty
with open('resul_permut_helth_corrected.json', 'r') as fp:
    permut_helth = json.load(fp)
res_permut_helth = json.loads(permut_helth) 

In [ ]:
# Load data pathologic
with open('resul_permut_pathologic_corrected.json', 'r') as fp:
    permut_pathologic = json.load(fp)
res_permut_pathologic = json.loads(permut_pathologic) 

In [ ]:
len(res_permut_pathologic)

# Comparing results

In [ ]:
distTest(20,5,4,res_permut_helth[0:20],'healty','L1',col='#9addf4')

In [ ]:
distTest(15,5,3,res_permut_pathologic[0:15],'Pathologic','L1',col='#e94975')

In [ ]:
healthy_summary = compareTest(res_permut_helth)

In [ ]:
pathologic_summary = compareTest(res_permut_pathologic)

In [ ]:
threshold0=0.9980001
thresholdn=1.00170625
thresholdsd=0.00088248
reff=0.01
checkH = checkHCP(res_permut_helth,threshold0,thresholdn,thresholdsd,reff)
checkP = checkHCP(res_permut_pathologic,threshold0,thresholdn,thresholdsd,reff)

In [ ]:
np.unique(checkH, return_counts=True)

In [ ]:
np.unique(checkP, return_counts=True)

## Detect label with high r-permut power

In [ ]:
l_low = np.median(healthy_summary['d_q005'])
l_upp = np.median(healthy_summary['d_q995'])
true_class,false_class,all_labels = detectLabelHigh(pathologic_summary['d_q5'],l_low,l_upp)
print('Low: ',l_low)
print('Upper: ',l_upp)

In [ ]:
n_true = Counter(true_class)
n_false = Counter(false_class)

true_key = n_true.keys()
false_key = n_false.keys()
power_detect = {}
for k in true_key:
    if(k in false_key):
        power_detect[k] = {'true':n_true[k],'false':n_false[k],'true_prop':np.round(n_true[k]/(n_true[k]+n_false[k]),3)}
    else:
        power_detect[k] = {'true':n_true[k],'false':0,'true_prop':1}

In [ ]:
power_detect

# rpermut whitn non-zero pixels

In [ ]:
resul_permut_helth_nonZero=[]
resul_permut_pathologic_nonZero=[]

In [ ]:
%%time
for i in np.arange(len(dat_healt)):
    res = rPermut2(500,dat_healt[i]['left'],dat_healt[i]['right'],1,n,'rate')
    Resh = {'R':res['R'],'label':dat_healt[i]['label'],'norm_x':res['norm_x'],'norm_y':res['norm_y']}
    resul_permut_helth_nonZero.append(Resh)
    

In [ ]:
%%time
for i in np.arange(len(dat_pathologic)):
    res = rPermut2(500,dat_pathologic[i]['left'],dat_pathologic[i]['right'],1,n,'rate')
    Resp = {'R':res['R'],'label':dat_pathologic[i]['label'].tolist(),'norm_x':res['norm_x'],'norm_y':res['norm_y']}
    resul_permut_pathologic_nonZero.append(Resp)

In [ ]:
print('health neuroimaging test: ',len(resul_permut_helth_nonZero))
print('pathologic neuroimaging test: ',len(resul_permut_pathologic_nonZero))

In [ ]:
# Save healty
dumped_healty = json.dumps(resul_permut_helth_nonZero, cls=NumpyEncoder) # , sort_keys=False, indent=4

with open('resul_permut_helth_nonZero.json', 'w') as f:
    json.dump(dumped_healty, f) # , sort_keys=False, indent=4

In [ ]:
# Save patholigic
dumped_pathologic = json.dumps(resul_permut_pathologic_nonZero, cls=NumpyEncoder) # , sort_keys=False, indent=4

with open('resul_permut_pathologic_nonZero.json', 'w') as f:
    json.dump(dumped_pathologic, f) # , sort_keys=False, indent=4

In [ ]:
distTest(20,5,4,resul_permut_helth_nonZero[0:20],'healty','L1')

In [ ]:
distTest(20,5,4,resul_permut_pathologic_nonZero[0:20],'pathologic','L1')

In [ ]:
healthy_summary_nonZero = compareTest(resul_permut_pathologic_nonZero)

In [ ]:
patholog_summary_nonZero = compareTest(resul_permut_helth_nonZero)